# Batch Analysis for hawk/dove simulation with multiple risk attitudes, no adjustment

- What % of time do agents play Hawk, by risk attitude? risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?
- Cumulative wealth analysis by risk attitude

To track how often agents play Hawk, we need to collect data for every round.

This data was generated by running:
```console
simulatingrisk/hawkdovemulti/batch_run.py --params no_adjustment --agent-data --collect-data every_round
```

Each row in the data file represents one round for each agent.

In [1]:
import pandas as pd

# df = pd.read_csv("../../data/hawkdovemulti/2025-07-22T170057_747737_agent.csv")
df = pd.read_csv("../../data/hawkdovemulti/2025-07-23T135510_859267_agent.csv")
# code still uses risk_level internally, but relabel as risk attitude
df = df.rename(columns={'risk_level': 'risk_attitude'})
# drop the risk_level_changed, since it is not relevant here (no adjustment = no changes)
df = df.drop("risk_level_changed", axis=1)
# add a numeric field to turn choice of play to 1/0 hawk, for aggregation
df['played_hawk'] = df.choice.apply(lambda x: 1 if x == 'hawk' else 0)
df.head()

,RunId,iteration,Step,AgentID,risk_attitude,choice,points,played_hawk
0,0,0,1,0,0,dove,11,0
1,0,0,1,1,0,dove,10,0
2,0,0,1,2,0,hawk,12,1
3,0,0,1,3,0,dove,12,0
4,0,0,1,4,0,dove,11,0


## Percent of the time agents play Hawk, by risk attitude

What % of time do risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?

- Guess from observation is is >90% for R=1, <10% for R8, but we want to have statistics for this: for X trials, how many of them does R1 play Hawk more than 90% of the time?
- Also useful to have statistics e.g. R=2 played Hawk between 80-90% of the time, or whatever the result is.


In [2]:
# each row in the data frame is a play by an agent on the grid
# group by risk level, then:
# - count the number of rows 
# - sum the played_hawk field (= number of times played hawk)

hawk_by_risk_attitude = df.groupby("risk_attitude", as_index=False)["played_hawk"].agg(["count", "sum"])

# use aggregate values to calculate % of time agents by risk attitude played hawk based on the count and sum
hawk_by_risk_attitude["pct_play_hawk"] = hawk_by_risk_attitude.apply(lambda x: (x["sum"] / x["count"]) * 100, axis=1)

hawk_by_risk_attitude

,risk_attitude,count,sum,pct_play_hawk
0,0,2365102,2331683,98.586995
1,1,2358948,2270929,96.268718
2,2,2386695,2284240,95.707244
3,3,2386017,2004107,83.993827
4,4,2398564,1657126,69.088254
5,5,2403275,733976,30.540658
6,6,2373816,385246,16.228975
7,7,2376684,102162,4.298510
8,8,2363429,85989,3.638315
9,9,2355395,33520,1.423116


In [3]:
(hawk_by_risk_attitude[["risk_attitude", "pct_play_hawk"]]).style \
  .format(precision=1, thousands=".", decimal=".") \
  .relabel_index(["Risk Attitude", "% Hawk"], axis=1) \
  .set_caption("% of time agents play Hawk") \
  .format(lambda x: f"{x:.1f}%", subset='pct_play_hawk') \
  .hide()   # hide the index



Risk Attitude,% Hawk
0,98.6%
1,96.3%
2,95.7%
3,84.0%
4,69.1%
5,30.5%
6,16.2%
7,4.3%
8,3.6%
9,1.4%


In [4]:
import altair as alt

alt.Chart(hawk_by_risk_attitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_play_hawk", title="% time plays Hawk")
).properties(title="% of time agents play Hawk")

alt.Chart(...)

Context: how many runs are these numbers drawn from?



In [6]:
len(df.RunId.unique())

total_unique_runs = len(df.RunId.unique())
total_iterations = len(df.iteration.unique())
n_combinations = int(total_unique_runs / total_iterations)

# Step is the round count indicator
longest_run = df.Step.max()    # highest across all runs
# average of max value for each run
average_run = df.groupby('RunId')['Step'].max().mean()

print(f"""{total_unique_runs:,} total unique runs; {total_iterations} iterations of {n_combinations} different parameter combinations.

Longest run: {longest_run} steps
Average run: {average_run:.1f} steps
""")

2,700 total unique runs; 100 iterations of 27 different parameter combinations.

Longest run: 71 steps
Average run: 39.2 steps



### Analysis filtered by simulation parameters

In [16]:
# identify the last round of each run
# for both wealth analysis and model parameters, we want to look at the last round (Step) of each run

last_round_df = df.groupby('RunId', as_index=False)['Step'].max()
last_round_df.head()

,RunId,Step
0,0,31
1,1,57
2,2,52
3,3,59
4,4,31


In [39]:
# load model data and filter to last round for each run
full_model_df = pd.read_csv("../../data/hawkdovemulti/2025-07-23T135510_859267_model.csv")
model_df = pd.merge(last_round_df, full_model_df, on=['RunId', 'Step'], how="left")
# limit to only those fields that are needed for our analysis
model_df = model_df[["RunId", "iteration", "risk_distribution", "play_neighborhood", "observed_neighborhood", "grid_size"]]
model_df.head()

,RunId,iteration,risk_distribution,play_neighborhood,observed_neighborhood,grid_size
0,0,0,uniform,8,8,5
1,1,1,uniform,8,8,5
2,2,2,uniform,8,8,5
3,3,3,uniform,8,8,5
4,4,4,uniform,8,8,5


In [35]:
print(f"""Simulation parameters:

Grid size: {', '.join(str(n) for n in sorted(model_df.grid_size.unique()))}
Play neighborhood size: {', '.join(str(n) for n in sorted(model_df.play_neighborhood.unique()))}
Observed neighborhood sized: {', '.join(str(n) for n in sorted(model_df.observed_neighborhood.unique()))}

""")

Simulation parameters:

Grid size: 5, 10, 25
Play neighborhood size: 4, 8, 24
Observed neighborhood sized: 4, 8, 24




In [52]:
# merge agent data with model data so we can filter by starting parameters
agent_df_params = pd.merge(df, model_df, on=["RunId", "iteration"], how="left")


### Percent of time playing Hawk by grid size


In [53]:
hawk_by_gridsize_riskattitude = agent_df_params.groupby(["grid_size", "risk_attitude"], as_index=False)["played_hawk"].agg(["count", "sum"])

# use aggregate values to calculate % of time agents by risk attitude played hawk based on the count and sum
hawk_by_gridsize_riskattitude["pct_play_hawk"] = hawk_by_gridsize_riskattitude.apply(lambda x: (x["sum"] / x["count"]) * 100, axis=1)

alt.Chart(hawk_by_gridsize_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_play_hawk", title="% time plays Hawk")
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

### Percent of time playing Hawk by play neighborhood


In [54]:
hawk_by_playnhood_riskattitude = agent_df_params.groupby(["play_neighborhood", "risk_attitude"], as_index=False)["played_hawk"].agg(["count", "sum"])

# use aggregate values to calculate % of time agents by risk attitude played hawk based on the count and sum
hawk_by_playnhood_riskattitude["pct_play_hawk"] = hawk_by_playnhood_riskattitude.apply(lambda x: (x["sum"] / x["count"]) * 100, axis=1)

alt.Chart(hawk_by_playnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_play_hawk", title="% time plays Hawk")
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

### Percent of time playing Hawk by observed neighborhood


In [55]:
hawk_by_obsnhood_riskattitude = agent_df_params.groupby(["observed_neighborhood", "risk_attitude"], as_index=False)["played_hawk"].agg(["count", "sum"])

# use aggregate values to calculate % of time agents by risk attitude played hawk based on the count and sum
hawk_by_obsnhood_riskattitude["pct_play_hawk"] = hawk_by_obsnhood_riskattitude.apply(lambda x: (x["sum"] / x["count"]) * 100, axis=1)

alt.Chart(hawk_by_obsnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_play_hawk", title="% time plays Hawk")
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

## Cumulative Wealth analysis


- mean and quartiles for wealth by R
- does mean vary between Rs or is it roughly the same?
  - quartiles look different, but need a statistic; esp. compare lower-R quartile to higher-R quartile
     - expect/hope that lower quartile is higher for R1 than R8, higher quartile is higher for R8 than R1
     - what's going on in the middle?


In [59]:
# combine the last round dataframe and full agent dataframe to get just the last round
agents_last_round_df = pd.merge(last_round_df, df, on=['RunId', 'Step'], how="left")

# hard to compare points across runs, since it depends on how long the simulation ran; scale points by number of runs
agents_last_round_df['scaled_points'] = agents_last_round_df.apply(lambda x: (x['points'] / x['Step'])*10, axis=1)

# merge with model parameters, for filtering
agents_last_round_df = pd.merge(agents_last_round_df, model_df, on=["RunId", "iteration"])

agents_last_round_df.head(10)

,RunId,Step,iteration,AgentID,risk_attitude,choice,points,played_hawk,scaled_points,risk_distribution,play_neighborhood,observed_neighborhood,grid_size
0,0,31,0,0,0,hawk,281,1,90.645161,uniform,8,8,5
1,0,31,0,1,0,hawk,370,1,119.354839,uniform,8,8,5
2,0,31,0,2,0,hawk,369,1,119.032258,uniform,8,8,5
3,0,31,0,3,0,hawk,459,1,148.064516,uniform,8,8,5
4,0,31,0,4,0,hawk,458,1,147.741935,uniform,8,8,5
5,0,31,0,5,5,hawk,387,1,124.838710,uniform,8,8,5
6,0,31,0,6,8,dove,312,0,100.645161,uniform,8,8,5
7,0,31,0,7,6,dove,327,0,105.483871,uniform,8,8,5
8,0,31,0,8,1,hawk,369,1,119.032258,uniform,8,8,5
9,0,31,0,9,2,hawk,548,1,176.774194,uniform,8,8,5


In [64]:
# our data has a lot of rows; enable vegafusion so altair can calculate quartiles for us
alt.data_transformers.enable("vegafusion")

alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("points", title="Wealth "),
)

alt.Chart(...)

In [62]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("scaled_points", title="Wealth "),
).properties(title="Cumulative wealth, scaled by simulation length")

alt.Chart(...)

In [69]:
# what if we look at wealth at round 31 across simulations?

agents_round31_df = df[df['Step'] == 31]

alt.Chart(agents_round31_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("points", title="Wealth"),
).properties(title="Cumulative wealth at round 31 across runs")

alt.Chart(...)

In [15]:
# Altair boxplot is calculating quartiles for us, but we can calculate them directly as well

# Q1: 25th percentile
def q1(x):
    return x.quantile(0.25)

# Q2: 50th percentile
def q2(x):
    return x.quantile(0.50)

# Q3: 75th percentile
def q3(x):
    return x.quantile(0.75)


wealth_by_risk_attitude = agents_last_round_df.groupby("risk_attitude", as_index=False)["points"].agg(["mean", "min", "max", q1, q2, q3]) 
wealth_by_risk_attitude

,risk_attitude,mean,min,max,q1,q2,q3
0,0,661.518227,0,3324,276.0,432.0,1071.0
1,1,660.854997,0,3294,275.0,424.0,1070.0
2,2,661.653835,0,3190,275.0,429.0,1071.0
3,3,654.602284,0,3083,273.0,416.0,1066.0
4,4,649.914763,0,3105,273.0,388.0,1054.0
5,5,636.167965,4,3296,242.0,373.0,1069.0
6,6,634.330700,70,3322,219.0,373.0,1079.0
7,7,631.916691,106,2734,216.0,373.0,1085.0
8,8,631.096809,120,2742,216.0,373.0,1085.0
9,9,629.906388,120,2734,216.0,373.0,1085.0


In [73]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title="Cumulative wealth, scaled by simulation length")

alt.FacetChart(...)

In [74]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")
).properties(title="Cumulative wealth, scaled by simulation length")

alt.FacetChart(...)

In [76]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude"),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")
).properties(title="Cumulative wealth, scaled by simulation length")

alt.FacetChart(...)